In [1]:
import os
import gc
import math
import json
from datetime import datetime
import shutil
from pathlib import Path
import optuna
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from tqdm import tqdm
import torch
from sklearn.model_selection import KFold, GroupKFold, StratifiedGroupKFold
from torch.utils.data import DataLoader
from transformers import BatchEncoding, BertTokenizerFast
from typing import Dict, List, Tuple, NamedTuple
import scml
import mylib

In [2]:
class Conf(NamedTuple):
    sample_frac: float = 1
    epochs: int = 2
    lr: Tuple[float, float] = (1e-3, 1e-3)
    embedding_size: int = 32  # 64 hits oom
    negative_samples: int = 10
    batch_size: int = 2048
    model_name: str = "word2vec"
    gradient_checkpointing: bool = False
    gpus: List[int] = [0]
    patience: int = 0
    n_trials: int = 1
    n_folds: int = 3
    seed: int = 31
    
        
conf = Conf()
print(conf)

Conf(sample_frac=1, epochs=2, lr=(0.001, 0.001), embedding_size=32, negative_samples=10, batch_size=2048, model_name='word2vec', gradient_checkpointing=False, gpus=[0], patience=0, n_trials=1, n_folds=3, seed=31)


In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(conf.seed)
pl.seed_everything(conf.seed)

Global seed set to 31


31

In [4]:
device: torch.device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce GTX 1060 6GB
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [5]:
with open("input/vocab2.json") as f:
    id2label = json.load(f)
print(f"len(id2label)={len(id2label):,}\nid2label[:10]={id2label[:10]}")

len(id2label)=3,748,278
id2label[:10]=[['1460571_click', 121287], ['108125_click', 114456], ['29735_click', 101148], ['485256_click', 97154], ['1733943_click', 91395], ['184976_click', 85122], ['832192_click', 81127], ['1502122_click', 73805], ['554660_click', 72161], ['1603001_click', 68519]]


In [6]:
df = pd.read_parquet("input/pairs.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14461968 entries, 0 to 14461967
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   center   int32
 1   outside  int32
dtypes: int32(2)
memory usage: 110.3 MB


In [7]:
ds = mylib.SkipGramDataset(
    center_words=df["center"].tolist(), 
    outside_words=df["outside"].tolist(),
    #session_ids=sids,
)
print(f"len(ds)={len(ds):,}\n{ds[30]}\n{ds[0]}")

len(ds)=14,461,968
{'center_words': tensor(154), 'outside_words': tensor(101)}
{'center_words': tensor(3), 'outside_words': tensor(130483)}


In [8]:
del df
gc.collect()
job_ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = Path("models") / conf.model_name / job_ts
job_dir.mkdir(parents=True, exist_ok=True)
print(f"job_dir={job_dir}")

job_dir=models\word2vec\20230117_073345


# Train final model on best Hps

In [9]:
%%time
#splitter = GroupKFold(n_splits=50)
splitter = KFold(n_splits=50)
dummy = np.zeros(len(ds))
#for ti, vi in splitter.split(dummy, y=ds.stratification(), groups=ds.groups()):
for ti, vi in splitter.split(dummy):
    tra_ds = torch.utils.data.Subset(ds, ti)
    val_ds = torch.utils.data.Subset(ds, vi)
    break
print(f"len(tra_ds)={len(tra_ds)}, len(val_ds)={len(val_ds)}")

len(tra_ds)=14172728, len(val_ds)=289240
Wall time: 65.5 ms


In [10]:
#best = df.iloc[0].to_dict()
best = {
    "lr": conf.lr[0],
}
print(f"best={best}")

best={'lr': 0.001}


In [11]:
model = mylib.SkipGramWord2Vec(
    lr=best["lr"],
    vocab_size=len(id2label),
    embedding_size=conf.embedding_size,
    negative_samples=conf.negative_samples,
)
print(model)

SkipGramWord2Vec(
  (embeddings): Embedding(3748278, 32)
)


In [12]:
gc.collect()
torch.cuda.empty_cache()
trainer = pl.Trainer(
    default_root_dir=job_dir,
    accelerator="gpu", 
    devices=conf.gpus,
    max_epochs=conf.epochs,  
    callbacks=mylib.training_callbacks(patience=conf.patience),
    deterministic=False,
)
trainer.fit(
    model,
    train_dataloaders=DataLoader(
        tra_ds,
        batch_size=conf.batch_size,
        shuffle=True,
        num_workers=0,
    ),
    val_dataloaders=DataLoader(
        val_ds,
        batch_size=conf.batch_size,
        shuffle=False,
        num_workers=0,
    ),
)

s:\dev\seahrh\kaggle-otto-recommender-system-2022\env\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: models\word2vec\20230117_073345\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | embeddings | Embedding | 119 M 
-----------------------------------------
119 M     Trainable params
0         Non-trainable params
119 M     Total params
479.780   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

s:\dev\seahrh\kaggle-otto-recommender-system-2022\env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
s:\dev\seahrh\kaggle-otto-recommender-system-2022\env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:241: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 7.624
Epoch 0, global step 6921: 'val_loss' reached 7.62381 (best 7.62381), saving model to 'models\\word2vec\\20230117_073345\\lightning_logs\\version_0\\checkpoints\\epoch=0-step=6921.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 1 records. Best score: 7.624. Signaling Trainer to stop.
Epoch 1, global step 13842: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=2` reached.


In [13]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")
print(f"Saved {str(job_dir)}")

Total time taken 1:01:48.789389
Saved models\word2vec\20230117_073345
